## Elementary Effects for the Rust Model

In [2]:
import os
import numpy as np
import pandas as pd
from functools import partial
from econsa.morris import elementary_effects
# from python.auxiliary import rust_model_morris

In [2]:
def rust_model_morris(x, n_draws, trans_probs, init_dict_estimation, demand_dict):
    
    n_evaluations = n_draws.copy()
    
    # Adapt function to work with changed number of trans_probs as well.
    n_trans_probs = len(trans_probs)
    
    demand_inputs = np.zeros((n_evaluations, n_trans_probs +2))
    demand_inputs[:, :n_trans_probs] = trans_probs
    demand_inputs[:, n_trans_probs:] = x[:, :]
    
    demand_output = np.zeros((n_evaluations, 1))
    
    get_demand_partial = partial(get_demand, init_dict=init_dict_estimation, demand_dict=demand_dict)
    def _get_demand_mapping(x):
        return get_demand_partial(demand_params=x).iloc[0]['demand']
    
    demand_output = np.array(list(map(_get_demand_mapping, demand_inputs)))
    
    return demand_output

In [3]:
# Get model.

# Set simulating variables.
disc_fac = 0.9999
num_buses = 50
num_periods = 120  # Ten years.
gridsize = 1000  # For plotting.
# We use the cost parameters and transition probabilities from the replication.
params = np.array([10.07780762, 2.29417622])  # Table X in Rust (1987): 11.7257, 2.4569
# Old values used before February 15th: 0.39189182, 0.59529371, 0.01281447
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
#trans_probs = np.array([0.0937, 0.4475, 0.4459, 0.0127])  # Four in table X of Rust (1987): 0.0937, 0.4475, 0.4459, 0.0127
scale = 1e-3

num_states = 175

init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

demand_dict = {
    "RC_lower_bound": 11.5,
    "RC_upper_bound": 11.5,
    "demand_evaluations": 1,
    "tolerance": 1e-10,
    "num_periods": num_periods,
    "num_buses": num_buses,
}

In [4]:
n_draws = 10

cov_data = np.load(os.path.join('data', 'cov_rc_theta_11.npy'))
mean = np.load(os.path.join('data', 'mean_rc_theta_11.npy'))

params = pd.DataFrame(data=mean, columns=['value'])
cov = pd.DataFrame(data=cov_data)

model = partial(
    rust_model_morris, n_draws=n_draws, trans_probs=trans_probs, 
    init_dict_estimation=init_dict_estimation, demand_dict=demand_dict,
    )

In [ ]:
%%time

n_cores = 5

elementary_effects(model, params, cov, n_draws, 'sobol', n_cores)

In [2]:
def model_func(params):
    return 5

names = ["x1", "x2", "x3"]
params = pd.DataFrame(columns=["value"], data=[0, 0.0, 0], index=names)

cov = pd.DataFrame(
    data=[[1, 0.9, 0.4], [0.9, 1, 0.01], [0.4, 0.01, 1]],
    columns=names,
    index=names,
)
n_draws = 100

elementary_effects(model_func, params, cov, n_draws)

NameError: name 'model_func' is not defined